In [1]:
from src.helper_functions import xor_hex_strings, single_byte_xor, repeating_key_xor, xor_bytes, hamming_distance, output_repeated_block, has_repeated_blocks, find_block
from src.file_loading import load_file_as_bytes, load_file_as_b64
from src.conversions import int2bytes, hex2bytes, string2bytes, hex2b64, string2dict
from src.symmetric_encryption import is_ecb_mode, aes128_ecb_encrypt, aes128_ecb_decrypt, aes128_cbc_encrypt, aes128_cbc_decrypt, determine_block_size
from src.padding import apply_pkcs_7, remove_pkcs_7
# 
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import secrets

## **Challenge 17: PKCS#7 padding validation**

In [2]:
# Assume that key is unknown but same for all encryptions
encryption_oracle_17_key = secrets.token_bytes(16)

In [3]:
def encryption_oracle_17():
    xxxx = [
        b"MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
        b"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
        b"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
        b"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
        b"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
        b"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
        b"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
        b"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
        b"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
        b"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93",
    ]



    chose = secrets.choice(xxxx)
    plaintext = apply_pkcs_7(base64.b64decode(chose), 16)
    IV = secrets.token_bytes(16)
    return aes128_cbc_encrypt(plaintext, encryption_oracle_17_key, IV), IV


def correct_padding_oracle(ciphertext, IV) -> bool:
    plaintext = aes128_cbc_decrypt(ciphertext, encryption_oracle_17_key, IV)
    try:
        remove_pkcs_7(plaintext)
        return True
    except Exception:
        return False

In [4]:
cyphertext, IV = encryption_oracle_17()
correct_padding_oracle(cyphertext, IV)

True

In [5]:
p = b''
# for i in range(1, 16):
#     # p[-1] ^ 0x3D = 0x01, where p is the last byte of the plaintext
#     # Therefore p[-1] = 0x01 ^ 0x3D = 0x3C
#     for j in range(0, 256):
#         new_IV = IV[:-i-1] + int2bytes(j)
#         if correct_padding_oracle(cyphertext[0:16], new_IV):
#             p = int2bytes(j ^ 0x3C) + p


for j in range(0, 256):
    new_IV = IV[:-2] + int2bytes(j)
    if correct_padding_oracle(cyphertext[0:16], new_IV):
        p = int2bytes(j) + p





0x01 ^ 0x3D


AssertionError: The two byte arrays must have the same shape